# stream

In [ ]:
open rust_operators

()



In [ ]:
//// test

open testing

()



## types

In [ ]:
inl types () =
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"async_std::io::BufReader<$0>\")>]\n#endif\ntype async_std_io_BufReader<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::io::BufReader<$0>\")>]\n#endif\ntype std_io_BufReader<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::io::Error\")>]\n#endif\ntype std_io_Error = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::io::Lines<$0>\")>]\n#endif\ntype std_io_Lines<'T> = class end"

()



## io_error

In [ ]:
nominal io_error = $"std_io_Error"

()



## buf_reader

In [ ]:
nominal buf_reader t = $'std_io_BufReader<`t>'

()



## async_buf_reader

In [ ]:
nominal async_buf_reader t = $'async_std_io_BufReader<`t>'

()



## new_buf_reader

In [ ]:
inl new_buf_reader forall t. (x : t) : buf_reader t =
    !\($'"std::io::BufReader::new(!x)"')

()



## lines

In [ ]:
nominal lines t = $'std_io_Lines<`t>'

()



## buf_read_lines

In [ ]:
inl buf_read_lines forall t. (buf_reader : buf_reader t) : lines (buf_reader t) =
    !\($'"std::io::BufRead::lines(!buf_reader)"')

()



## buf_reader_read

In [ ]:
inl buf_reader_read forall t dim. (slice : am'.slice' dim dim) (buf_reader : buf_reader t) : resultm.result' dim io_error =
    !\($'"std::io::Read::read(!buf_reader, !slice)"')

()



## stream

In [ ]:
union rec stream t =
    | StreamCons : t * (() -> stream t)
    | StreamNil

()



## fold

In [ ]:
inl fold fn init s =
    inl rec body acc = function
        | StreamCons (st, fn') => loop (fn acc st) (fn' ())
        | StreamNil => acc
    and inl loop acc = join_body body acc
    loop init s

()



## fold_back

In [ ]:
inl fold_back fn s init =
    inl rec body acc = function
        | StreamCons (st, fn') => fn st (loop acc (fn' ()))
        | StreamNil => acc
    and inl loop acc = join_body body acc
    loop init s

()



## to_list

In [ ]:
inl to_list s =
    (s, [])
    ||> fold_back fun x acc =>
        x :: acc

()



## rev

In [ ]:
inl rev s =
    (StreamNil, s)
    ||> fold fun s x =>
        StreamCons (x, fun () => s)

()



## from_list

In [ ]:
inl from_list list =
    (list, StreamNil)
    ||> listm.foldBack fun x acc =>
        StreamCons (x, fun () => acc)

()



In [ ]:
//// test

listm.init 3i32 id
|> from_list
|> rev
|> to_list
|> _assert_eq [ 2; 1; 0 ]

type UH0 =
    | UH0_0
    | UH0_1 of int32 * UH0
let rec method0 () : unit =
    let v18 : string = "assert_eq"
    let v19 : UH0 = UH0_0
    let v20 : UH0 = UH0_1(0, v19)
    let v21 : UH0 = UH0_1(1, v20)
    let v22 : UH0 = UH0_1(2, v21)
    let v23 : UH0 = UH0_0
    let v24 : UH0 = UH0_1(0, v23)
    let v25 : UH0 = UH0_1(1, v24)
    let v26 : UH0 = UH0_1(2, v25)
    let v27 : string = $"{v18} / actual: %A{v22} / expected: %A{v26}"
    System.Console.WriteLine v27
    ()
method0()

assert_eq / actual: UH0_1 (2, UH0_1 (1, UH0_1 (0, UH0_0))) / expected: UH0_1 (2, UH0_1 (1, UH0_1 (0, UH0_0)))


## try_item

In [ ]:
inl try_item i s =
    inl rec body i = function
        | StreamCons (x, _) when i <= 0 => Some x
        | StreamCons (_, fn) => loop (i - 1) (fn ())
        | StreamNil => None
    and inl loop acc s' =
        match var_is acc, var_is s' with
        | false, false => body acc s'
        | _ =>
            inl acc = dyn acc
            inl s' = dyn s'
            join body acc s'
    loop i s

inl item i =
    try_item i >> optionm.value

()



In [ ]:
//// test

listm.init 10i32 id
|> from_list
|> item 9i32
|> _assert_eq 9

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{9} / expected: %A{9}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: 9 / expected: 9


## new_infinite_stream

In [ ]:
inl new_infinite_stream fn =
    inl rec loop n =
        StreamCons (fn n, fun () => loop (n + 1))
    loop 0

inl new_infinite_stream_ fn =
    let rec loop n =
        StreamCons (fn n, fun () => loop (n + 1))
    loop 0

()



In [ ]:
//// test

new_infinite_stream print_and_return
|> item 4i32
|> _assert_eq 4i32

let rec method0 () : unit =
    printfn $"print_and_return / x: {0}"
    printfn $"print_and_return / x: {1}"
    printfn $"print_and_return / x: {2}"
    printfn $"print_and_return / x: {3}"
    printfn $"print_and_return / x: {4}"
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{4} / expected: %A{4}"
    System.Console.WriteLine v1
    ()
method0()

print_and_return / x: 0
print_and_return / x: 1
print_and_return / x: 2
print_and_return / x: 3
print_and_return / x: 4
assert_eq / actual: 4 / expected: 4


## new_finite_stream

In [ ]:
inl new_finite_stream fn max =
    inl rec loop n =
        if n >= max
        then StreamNil
        else StreamCons (fn n, fun () => loop (n + 1))
    loop 0

()



## memoize

In [ ]:
union memoized_stream t =
    | NotComputed : () -> stream t
    | Computed : stream t

inl memoize s =
    inl rec body s =
        inl state = mut (NotComputed s)
        fun () =>
            match *state with
            | Computed x => x
            | NotComputed fn =>
                inl new_state =
                    match fn () with
                    | StreamNil => StreamNil
                    | StreamCons (x, fn) => StreamCons (x, loop fn)
                state <- Computed new_state
                new_state
    and inl loop s' = join_body_unit body s s'
    loop (fun () => s)

()



In [ ]:
//// test

inl memo_stream = new_finite_stream print_and_return 10 |> memoize

memo_stream ()
|> item 3i32
|> _assert_eq 3i32

memo_stream ()
|> item 5i32
|> _assert_eq 5i32

type UH0 =
    | UH0_0 of int32 * (unit -> UH0)
    | UH0_1
and [<Struct>] US0 =
    | US0_0 of f0_0 : (unit -> UH0)
    | US0_1 of f1_0 : UH0
and Mut0 = {mutable l0 : US0}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int32
    | US1_1
let rec closure10 () () : UH0 =
    UH0_1
and closure9 () () : UH0 =
    printfn $"print_and_return / x: {9}"
    let v0 : (unit -> UH0) = closure10()
    UH0_0(9, v0)
and closure8 () () : UH0 =
    printfn $"print_and_return / x: {8}"
    let v0 : (unit -> UH0) = closure9()
    UH0_0(8, v0)
and closure7 () () : UH0 =
    printfn $"print_and_return / x: {7}"
    let v0 : (unit -> UH0) = closure8()
    UH0_0(7, v0)
and closure6 () () : UH0 =
    printfn $"print_and_return / x: {6}"
    let v0 : (unit -> UH0) = closure7()
    UH0_0(6, v0)
and closure5 () () : UH0 =
    printfn $"print_and_return / x: {5}"
    let v0 : (unit -> UH0) = closure6()
    UH0_0(5, v0)
and closure4 () () : UH0 =
    printfn $"print_and_return / x: {4}"
    let v0 : (unit -> UH0) = c

In [ ]:
//// test

inl memo_stream = new_infinite_stream_ print_and_return |> memoize

memo_stream ()
|> item 3i32
|> _assert_eq 3i32

memo_stream ()
|> item 5i32
|> _assert_eq 5i32

type UH0 =
    | UH0_0 of int32 * (unit -> UH0)
    | UH0_1
and [<Struct>] US0 =
    | US0_0 of f0_0 : (unit -> UH0)
    | US0_1 of f1_0 : UH0
and Mut0 = {mutable l0 : US0}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int32
    | US1_1
let rec closure0 (v0 : int32) () : UH0 =
    let v1 : int32 = v0 + 1
    method1(v1)
and method1 (v0 : int32) : UH0 =
    printfn $"print_and_return / x: {v0}"
    let v1 : (unit -> UH0) = closure0(v0)
    UH0_0(v0, v1)
and closure1 (v0 : UH0) () : UH0 =
    v0
and closure2 (v0 : UH0, v1 : Mut0) () : UH0 =
    let v2 : US0 = v1.l0
    match v2 with
    | US0_1(v3) -> (* Computed *)
        v3
    | US0_0(v4) -> (* NotComputed *)
        let v5 : UH0 = v4 ()
        let v12 : UH0 =
            match v5 with
            | UH0_0(v7, v8) -> (* StreamCons *)
                let v9 : (unit -> UH0) = method2(v0, v8)
                UH0_0(v7, v9)
            | UH0_1 -> (* StreamNil *)
                UH0_1
        let v13 : US0 = US0_1(v12)
        v1.l0 <- v13
  

## unfold

In [ ]:
inl unfold f x0 =
    inl rec body x =
        match f x with
        | Some (x', y) => StreamCons (x', fun () => loop y)
        | None => StreamNil
    and inl loop x = join_body_unit body x0 x
    loop x0

()



## iterate

In [ ]:
inl iterate f =
    unfold (fun x => Some (x, f x))

()



In [ ]:
//// test

iterate ((*) 2) 1i32
|> item 10i32
|> _assert_eq 1024

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{1024} / expected: %A{1024}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: 1024 / expected: 1024


## take_while

In [ ]:
inl take_while cond s =
    inl rec body i = function
        | StreamCons (st, fn) when cond st i => StreamCons (st, fun () => loop (i + 1) (fn ()))
        | _ => StreamNil
    and inl loop i = join_body body i
    loop 0 s

()



## sum

In [ ]:
inl sum seq =
    seq |> fold (+) 0

()



In [ ]:
//// test

listm.init 10i32 id
|> from_list
|> sum
|> _assert_eq 45

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{45} / expected: %A{45}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: 45 / expected: 45


In [ ]:
//// test

new_finite_stream print_and_return 10i32
|> take_while (fun n (_ : i32) => n < 5)
|> sum
|> _assert_eq 10

let rec method0 () : unit =
    printfn $"print_and_return / x: {0}"
    printfn $"print_and_return / x: {1}"
    printfn $"print_and_return / x: {2}"
    printfn $"print_and_return / x: {3}"
    printfn $"print_and_return / x: {4}"
    printfn $"print_and_return / x: {5}"
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{10} / expected: %A{10}"
    System.Console.WriteLine v1
    ()
method0()

print_and_return / x: 0
print_and_return / x: 1
print_and_return / x: 2
print_and_return / x: 3
print_and_return / x: 4
print_and_return / x: 5
assert_eq / actual: 10 / expected: 10


In [ ]:
//// test

new_infinite_stream print_and_return
|> take_while (fun n (_ : i32) => n < 5i32)
|> sum
|> _assert_eq 10

let rec method0 () : unit =
    printfn $"print_and_return / x: {0}"
    printfn $"print_and_return / x: {1}"
    printfn $"print_and_return / x: {2}"
    printfn $"print_and_return / x: {3}"
    printfn $"print_and_return / x: {4}"
    printfn $"print_and_return / x: {5}"
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{10} / expected: %A{10}"
    System.Console.WriteLine v1
    ()
method0()

print_and_return / x: 0
print_and_return / x: 1
print_and_return / x: 2
print_and_return / x: 3
print_and_return / x: 4
print_and_return / x: 5
assert_eq / actual: 10 / expected: 10


In [ ]:
//// test

iterate ((*) 6) 1i32
|> take_while (fun _ i => i <= 7i32)
|> to_list
|> _assert_eq [ 1i32; 6; 36; 216; 1296; 7776; 46656; 279936 ]

type UH0 =
    | UH0_0
    | UH0_1 of int32 * UH0
let rec method0 () : unit =
    let v63 : string = "assert_eq"
    let v64 : UH0 = UH0_0
    let v65 : UH0 = UH0_1(279936, v64)
    let v66 : UH0 = UH0_1(46656, v65)
    let v67 : UH0 = UH0_1(7776, v66)
    let v68 : UH0 = UH0_1(1296, v67)
    let v69 : UH0 = UH0_1(216, v68)
    let v70 : UH0 = UH0_1(36, v69)
    let v71 : UH0 = UH0_1(6, v70)
    let v72 : UH0 = UH0_1(1, v71)
    let v73 : UH0 = UH0_0
    let v74 : UH0 = UH0_1(279936, v73)
    let v75 : UH0 = UH0_1(46656, v74)
    let v76 : UH0 = UH0_1(7776, v75)
    let v77 : UH0 = UH0_1(1296, v76)
    let v78 : UH0 = UH0_1(216, v77)
    let v79 : UH0 = UH0_1(36, v78)
    let v80 : UH0 = UH0_1(6, v79)
    let v81 : UH0 = UH0_1(1, v80)
    let v82 : string = $"{v63} / actual: %A{v72} / expected: %A{v81}"
    System.Console.WriteLine v82
    ()
method0()

assert_eq / actual: UH0_1
  (1,
   UH0_1
     (6,
      UH0_1
        (36,
         UH0_1
           (216,
            UH0_1 (1296, UH

## indexed

In [ ]:
inl indexed s =
    ((StreamNil, 0), s)
    ||> fold fun (acc, i) x =>
        StreamCons ((i, x), fun () => acc), i + 1
    |> fst
    |> rev

()



In [ ]:
//// test

listm.init 10i32 ((*) 2)
|> from_list
|> indexed
|> item 5i32
|> _assert_eq (5i32, 10i32)

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{struct (5, 10)} / expected: %A{struct (5, 10)}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: struct (5, 10) / expected: struct (5, 10)


## map

In [ ]:
inl map fn s =
    (s, StreamNil)
    ||> fold_back fun x acc =>
        StreamCons (fn x, fun () => acc)

()



In [ ]:
//// test

listm.init 10i32 id
|> from_list
|> map ((*) 2)
|> item 5i32
|> _assert_eq 10i32

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{10} / expected: %A{10}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: 10 / expected: 10


## zip_with

In [ ]:
inl zip_with fn s1 s2 =
    inl rec loop s1 s2 =
        match s1, s2 with
        | StreamCons (st1, fn1), StreamCons (st2, fn2) =>
            StreamCons (fn st1 st2, fun () => loop (fn1 ()) (fn2 ()))
        | StreamNil, _ | _, StreamNil => StreamNil
    loop s1 s2

inl zip_with_ fn s1 s2 =
    let rec loop s1 s2 =
        match s1, s2 with
        | StreamCons (st1, fn1), StreamCons (st2, fn2) =>
            StreamCons (fn st1 st2, fun () => loop (fn1 ()) (fn2 ()))
        | StreamNil, _ | _, StreamNil => StreamNil
    loop s1 s2

()



In [ ]:
//// test

((listm.init 10i32 id |> from_list), (listm.init 10i32 ((*) 2) |> from_list))
||> zip_with (+)
|> item 2i32
|> _assert_eq 6

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{6} / expected: %A{6}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: 6 / expected: 6


## zip

In [ ]:
inl zip s1 s2 =
    zip_with pair s1 s2

inl zip_ s1 s2 =
    zip_with_ pair s1 s2

()



In [ ]:
//// test

((listm.init 10i32 id |> from_list), (listm.init 10i32 ((*) 2) |> from_list))
||> zip
|> item 5i32
|> _assert_eq (5, 10)

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{struct (5, 10)} / expected: %A{struct (5, 10)}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: struct (5, 10) / expected: struct (5, 10)


## unzip

In [ ]:
inl unzip s =
    inl rec body s =
        match s with
        | StreamCons ((x, y), fn) =>
            inl xs, ys = loop (fn ())
            StreamCons (x, fun () => xs), StreamCons (y, fun () => ys)
        | StreamNil => pair StreamNil StreamNil
    and inl loop x =
        if var_is x |> not
        then body x
        else
            inl x = dyn x
            join body x
    loop s

()



In [ ]:
//// test

listm.init 10i32 id
|> listm.map (fun x => x, x)
|> from_list
|> unzip
|> fun x, y =>
    x |> sum
    |> _assert_eq 45

    y |> sum
    |> _assert_eq 45

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{45} / expected: %A{45}"
    System.Console.WriteLine v1
    let v2 : string = $"{v0} / actual: %A{45} / expected: %A{45}"
    System.Console.WriteLine v2
    ()
method0()

assert_eq / actual: 45 / expected: 45
assert_eq / actual: 45 / expected: 45


## enumerate

In [ ]:
inl enumerate forall dim {int; number} t. (ar : a dim t) : a dim (unativeint * t) =
    open rust_operators
    inl (a ar) = ar
    inl ar = ar |> am'.to_vec
    !\($'"!ar.into_iter().enumerate().map(std::rc::Rc::new).collect()"')
    |> am'.vec_map from_pair
    |> am'.from_vec

()

